In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
from PIL import Image

import torch
import pprint
from torchvision import models, transforms
import tensorflow as tf

import random
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.layers import Conv2D, Dense, Reshape, Input, LSTM
from keras import Model, backend
import tensorflow as tf

import seaborn as sns


# stock_id=0をtime_idごとに画像変換する

In [ ]:
trade_ex = pd.read_parquet("../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=0")
book_ex = pd.read_parquet('../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=0')

In [ ]:
book_ex

In [ ]:
trade_ex

In [ ]:
samples = ["bid_price1","bid_price2","ask_price1","ask_price2"]

time_id = trade_ex["time_id"].unique()

for i,id in enumerate(time_id):
    plt.ioff()
    plt.figure(figsize=(20,5))
    
    book_test = book_ex[book_ex["time_id"]==id]
    trade_test = trade_ex[trade_ex["time_id"]==id]
    for num,a in enumerate(samples):
        if num<2:
            plt.plot(book_test["seconds_in_bucket"],book_test[a],label=a,color = 'green')
        else: 
            plt.plot(book_test["seconds_in_bucket"],book_test[a],label=a,color = 'red')

    plt.axis('off')
    plt.plot(trade_test["seconds_in_bucket"],trade_test["price"],label="trade_parquet",lw=10)
    plt.savefig(os.path.join('stock_'+str(i)+'.png')) #plt.saveig()でグラフを保存
    plt.close()

In [ ]:
from IPython.display import Image
Image("stock_0.png")

## 学習のための前処理〜学習

In [ ]:
from keras.preprocessing.image import load_img, img_to_array, array_to_img
from keras.applications.vgg16 import preprocess_input, VGG16, decode_predictions
import tensorflow as tf

from keras.preprocessing.image import load_img, img_to_array, array_to_img
from keras.preprocessing.image import random_rotation, random_shift, random_zoom
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.models import Sequential
from keras.models import model_from_json
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.utils import np_utils

In [ ]:
for i,id_ in enumerate(time_id):
    img_pil = load_img(f'stock_{i}.png', target_size=(224, 224))
    img = img_to_array(img_pil)
    img = img[tf.newaxis, ...]
    img_preprocessed = preprocess_input(img)
    if i == 0:
        img_array = img_preprocessed
    else:
        img_array = np.vstack((img_array, img_preprocessed))

In [ ]:
#ラベルのサンプル
y_train = np.random.rand(img_array.shape[0])
y_train

In [ ]:
img_array.shape

In [ ]:
##https://qiita.com/mako0715/items/b6605a77467ac439955bを参照

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(224, 224, 3)),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dense(1, activation='softmax')
])
# 任意のオプティマイザと損失関数を設定してモデルをコンパイルする
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# ネットワーク各層の出力内容を確認する
model.summary()
# モデルをトレーニングする
model.fit(img_array, y_train, epochs=5)

In [ ]:
def rmspe(y_true, y_pred):
    _=np.sqrt(np.mean(((y_pred - y_true) / y_true)**2))*100
    return _